# Práctica 1: Comparativa Avanzada de Optimizadores en MNIST

**Objetivo:** Estudiar el rendimiento de diferentes algoritmos de optimización (optimizers) sobre el problema de clasificación de dígitos manuscritos del dataset MNIST.

**Contenido:**
1. Preparación de datos con diferentes configuraciones
2. Implementación de múltiples optimizadores
3. Análisis comparativo detallado
4. Matrices de confusión
5. Curvas de aprendizaje
6. Métricas avanzadas

---

## 1. Importaciones y Configuración Inicial

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Configuración para reproducibilidad
tf.random.set_seed(42)
np.random.seed(42)

# Configuración de plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Configuración inicial completada")

## 2. Preparación Avanzada de Datos

In [ ]:
print("📋 Preparación de MNIST con Augmentación")
print("=" * 45)

# Cargar y preparar MNIST
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalizar a [0,1] y expandir dimensión
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Reshape para usar con CNN (28x28x1)
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# Crear split de validación
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.1, stratify=y_train, random_state=42
)

print(f"Datos de entrenamiento: {x_train.shape[0]} muestras")
print(f"Datos de validación: {x_val.shape[0]} muestras")
print(f"Datos de test: {x_test.shape[0]} muestras")
print(f"Clases: {len(np.unique(y_train))}")

# Data augmentation (opcional)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1
)

print("\n✅ Datos preparados con augmentación configurada")

## 3. Definición de Modelo Mejorado y Configuraciones de Optimizadores

In [ ]:
def create_improved_model(input_shape=(28, 28, 1), num_classes=10):
    """Modelo CNN mejorado con regularización"""
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2, 2),
        
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2, 2),
        
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Configuración de optimizadores con hiperparámetros ajustados
optimizers_config = {
    'SGD': tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True),
    'Adam': tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999),
    'RMSprop': tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9),
    'AdamW': tf.keras.optimizers.AdamW(learning_rate=0.001, weight_decay=1e-4),
    'Nadam': tf.keras.optimizers.Nadam(learning_rate=0.002),
    'Adagrad': tf.keras.optimizers.Adagrad(learning_rate=0.01)
}

# Configuración de entrenamiento
EPOCHS = 10
BATCH_SIZE = 128

print(f"🎆 {len(optimizers_config)} optimizadores configurados")
print(f"   Modelos CNN mejorados con regularización")
print(f"   Épocas: {EPOCHS}, Batch size: {BATCH_SIZE}")

## 4. Entrenamiento y Evaluación Comparativa

In [ ]:
results = {}
training_histories = {}

print("🔄 Iniciando entrenamiento comparativo...")
print("=" * 50)

for opt_name, optimizer in optimizers_config.items():
    print(f"\n⏱️  Entrenando con {opt_name}...")
    
    # Crear modelo fresco para cada optimizador
    model = create_improved_model()
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Entrenar con early stopping
    early_stop = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy', patience=3, restore_best_weights=True
    )
    
    start_time = time.time()
    history = model.fit(
        x_train, y_train,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(x_val, y_val),
        callbacks=[early_stop],
        verbose=0
    )
    training_time = time.time() - start_time
    
    # Evaluación
    train_loss, train_acc = model.evaluate(x_train, y_train, verbose=0)
    val_loss, val_acc = model.evaluate(x_val, y_val, verbose=0)
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    
    # Predicciones y matrices de confusión
    y_pred_train = np.argmax(model.predict(x_train, verbose=0), axis=1)
    y_pred_val = np.argmax(model.predict(x_val, verbose=0), axis=1)
    y_pred_test = np.argmax(model.predict(x_test, verbose=0), axis=1)
    
    cm_train = confusion_matrix(y_train, y_pred_train)
    cm_val = confusion_matrix(y_val, y_pred_val)
    cm_test = confusion_matrix(y_test, y_pred_test)
    
    # Errores por clase
    errors_per_class = {}
    for i in range(10):
        class_mask = (y_test == i)
        errors_per_class[i] = 1 - (y_pred_test[class_mask] == i).mean()
    
    # Almacenar resultados
    results[opt_name] = {
        'model': model,
        'training_time': training_time,
        'train_acc': train_acc,
        'val_acc': val_acc,
        'test_acc': test_acc,
        'train_loss': train_loss,
        'val_loss': val_loss,
        'test_loss': test_loss,
        'epochs_trained': len(history.history['loss']),
        'cm_train': cm_train,
        'cm_val': cm_val,
        'cm_test': cm_test,
        'errors_per_class': errors_per_class
    }
    
    training_histories[opt_name] = history.history
    
    print(f"   Tiempo: {training_time:.2f}s | Épocas: {len(history.history['loss'])} | Acc Test: {test_acc:.4f}")

print("\n✅ Entrenamiento completado para todos los optimizadores")

## 5. Visualización Comparativa Completa

In [ ]:
print("📈 Generando visualizaciones comparativas...")

# Preparar datos para visualización
opt_names = list(results.keys())

# Gráficos comparativos principales
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Análisis Comparativo de Optimizadores - MNIST', fontsize=16, fontweight='bold')

# 1. Accuracy Comparison
ax = axes[0, 0]
train_accs = [results[name]['train_acc'] for name in opt_names]
val_accs = [results[name]['val_acc'] for name in opt_names]
test_accs = [results[name]['test_acc'] for name in opt_names]

x = np.arange(len(opt_names))
width = 0.25

ax.bar(x - width, train_accs, width, label='Train', alpha=0.8)
ax.bar(x, val_accs, width, label='Validation', alpha=0.8)
ax.bar(x + width, test_accs, width, label='Test', alpha=0.8)

ax.set_xlabel('Optimizador')
ax.set_ylabel('Accuracy')
ax.set_title('Accuracy por Dataset')
ax.set_xticks(x)
ax.set_xticklabels(opt_names, rotation=45)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim(0.95, 1.0)

# 2. Training Time
ax = axes[0, 1]
times = [results[name]['training_time'] for name in opt_names]
bars = ax.bar(opt_names, times, color='coral', alpha=0.8)
ax.set_xlabel('Optimizador')
ax.set_ylabel('Tiempo (segundos)')
ax.set_title('Tiempo de Entrenamiento')
ax.grid(True, alpha=0.3)
plt.setp(ax.get_xticklabels(), rotation=45)
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{times[i]:.1f}s',
            ha='center', va='bottom')

# 3. Épocas entrenadas (con early stopping)
ax = axes[0, 2]
epochs = [results[name]['epochs_trained'] for name in opt_names]
bars = ax.bar(opt_names, epochs, color='lightgreen', alpha=0.8)
ax.set_xlabel('Optimizador')
ax.set_ylabel('Número de Épocas')
ax.set_title('Épocas Entrenadas')
ax.grid(True, alpha=0.3)
plt.setp(ax.get_xticklabels(), rotation=45)
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(epochs[i])}',
            ha='center', va='bottom')

# 4. Curvas de aprendizaje combinadas
ax = axes[1, 0]
for opt_name in opt_names[:3]:  # Solo top 3 para claridad
    history = training_histories[opt_name]
    ax.plot(history['accuracy'], label=f'{opt_name} Train', linestyle='--', alpha=0.7)
    ax.plot(history['val_accuracy'], label=f'{opt_name} Val', linewidth=2)

ax.set_xlabel('Época')
ax.set_ylabel('Accuracy')
ax.set_title('Curvas de Aprendizaje (Top 3)')
ax.legend()
ax.grid(True, alpha=0.3)

# 5. Loss comparison
ax = axes[1, 1]
test_losses = [results[name]['test_loss'] for name in opt_names]
bars = ax.bar(opt_names, test_losses, color='salmon', alpha=0.8)
ax.set_xlabel('Optimizador')
ax.set_ylabel('Loss')
ax.set_title('Test Loss Comparativo')
ax.grid(True, alpha=0.3)
plt.setp(ax.get_xticklabels(), rotation=45)

# 6. Efficiency plot (Accuracy vs Time)
ax = axes[1, 2]
for i, opt_name in enumerate(opt_names):
    ax.scatter(results[opt_name]['training_time'], 
               results[opt_name]['test_acc'],
               s=150, alpha=0.7, label=opt_name)

ax.set_xlabel('Tiempo de Entrenamiento (s)')
ax.set_ylabel('Test Accuracy')
ax.set_title('Eficiencia: Accuracy vs Tiempo')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📈 Visualizaciones generadas")

## 6. Tabla Resumen de Resultados

In [ ]:
# Crear tabla comparativa
comparison_data = []
for opt_name in opt_names:
    r = results[opt_name]
    comparison_data.append({
        'Optimizador': opt_name,
        'Train Acc': f"{r['train_acc']:.4f}",
        'Val Acc': f"{r['val_acc']:.4f}",
        'Test Acc': f"{r['test_acc']:.4f}",
        'Test Loss': f"{r['test_loss']:.4f}",
        'Tiempo (s)': f"{r['training_time']:.2f}",
        'Épocas': r['epochs_trained']
    })

df_comparison = pd.DataFrame(comparison_data)
df_comparison = df_comparison.sort_values('Test Acc', ascending=False)

print("📄 Tabla Resumen de Resultados")
print("=" * 50)
print(df_comparison.to_string(index=False))

# Identificar el mejor optimizador
best_optimizer = df_comparison.iloc[0]['Optimizador']
best_acc = df_comparison.iloc[0]['Test Acc']

print(f"\n🏆 Mejor optimizador: {best_optimizer}")
print(f"   Test Accuracy: {best_acc}")

# Calcular mejora promedio
worst_acc = float(df_comparison.iloc[-1]['Test Acc'])
best_acc_float = float(best_acc)
improvement = ((best_acc_float - worst_acc) / worst_acc) * 100

print(f"📈 Mejora del mejor vs peor: {improvement:.2f}%")

## 7. Matrices de Confusión Detalladas

In [ ]:
# Visualizar matrices de confusión para los top 3 optimizadores
top_3_optimizers = df_comparison['Optimizador'].head(3).values

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Matrices de Confusión - Top 3 Optimizadores (Test Set)', fontsize=14, fontweight='bold')

for i, opt_name in enumerate(top_3_optimizers):
    cm = results[opt_name]['cm_test']
    
    # Normalizar para mostrar porcentajes
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
    
    im = axes[i].imshow(cm_norm, interpolation='nearest', cmap=plt.cm.Blues)
    axes[i].set_title(f'{opt_name}\nTest Acc: {results[opt_name]["test_acc"]:.4f}')
    
    # Añadir texto en cada celda
    thresh = cm_norm.max() / 2.
    for j in range(cm.shape[0]):
        for k in range(cm.shape[1]):
            axes[i].text(k, j, f'{cm[j, k]}\n({cm_norm[j, k]:.1f}%)',
                        ha="center", va="center",
                        color="white" if cm_norm[j, k] > thresh else "black",
                        fontsize=8)
    
    axes[i].set_ylabel('Etiqueta Real')
    axes[i].set_xlabel('Predicción')
    axes[i].set_xticks(np.arange(10))
    axes[i].set_yticks(np.arange(10))

plt.tight_layout()
plt.show()

print("\n✅ Matrices de confusión generadas")

## 8. Análisis de Errores por Clase

In [ ]:
# Error rate por clase para cada optimizador
plt.figure(figsize=(12, 8))

# Crear heatmap de tasas de error
error_matrix = np.zeros((len(opt_names), 10))
for i, opt_name in enumerate(opt_names):
    for digit in range(10):
        error_matrix[i, digit] = results[opt_name]['errors_per_class'][digit] * 100

sns.heatmap(error_matrix, 
            xticklabels=[f'Dígito {i}' for i in range(10)],
            yticklabels=opt_names,
            annot=True, fmt='.2f', cmap='YlOrRd',
            cbar_kws={'label': 'Tasa de Error (%)'})

plt.title('Tasa de Error por Clase y Optimizador', fontsize=14, fontweight='bold')
plt.xlabel('Clases (Dígitos)')
plt.ylabel('Optimizadores')
plt.tight_layout()
plt.show()

# Identificar dígitos más difíciles
avg_errors = np.mean(error_matrix, axis=0)
difficult_digits = np.argsort(avg_errors)[::-1][:3]

print(f"\n🔴 Dígitos más difíciles de clasificar:")
for i, digit in enumerate(difficult_digits):
    print(f"   {i+1}. Dígito {digit}: {avg_errors[digit]:.2f}% error promedio")

print("\n✅ Análisis de errores por clase completado")

## 9. Curvas de Aprendizaje Detalladas

In [ ]:
# Curvas de aprendizaje detalladas
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for i, opt_name in enumerate(opt_names):
    history = training_histories[opt_name]
    
    # Subplot para cada optimizador
    ax = axes[i]
    
    # Accuracy curves
    ax.plot(history['accuracy'], 'b-', label='Train Accuracy', linewidth=2)
    ax.plot(history['val_accuracy'], 'r-', label='Val Accuracy', linewidth=2)
    
    # Loss curves (secondary y-axis)
    ax2 = ax.twinx()
    ax2.plot(history['loss'], 'b--', alpha=0.6, label='Train Loss')
    ax2.plot(history['val_loss'], 'r--', alpha=0.6, label='Val Loss')
    
    ax.set_title(f'{opt_name}\nFinal Test Acc: {results[opt_name]["test_acc"]:.4f}')
    ax.set_xlabel('Época')
    ax.set_ylabel('Accuracy', color='black')
    ax2.set_ylabel('Loss', color='gray')
    
    # Leyenda combinada
    lines1, labels1 = ax.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax.legend(lines1 + lines2, labels1 + labels2, loc='center right')
    
    ax.grid(True, alpha=0.3)

plt.suptitle('Curvas de Aprendizaje Detalladas por Optimizador', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n✅ Curvas de aprendizaje detalladas generadas")

## 10. Conclusiones y Recomendaciones

In [ ]:
print("📋 CONCLUSIONES DEL ESTUDIO COMPARATIVO")
print("=" * 55)

# Ranking final
print("1. RANKING POR ACCURACY EN TEST:")
for i, row in df_comparison.iterrows():
    print(f"   {i+1}. {row['Optimizador']}: {row['Test Acc']} ({row['Tiempo (s)']}s)")

# Eficiencia (accuracy/time)
print("\n2. EFICIENCIA (Accuracy/Tiempo):")
efficiencies = {}
for opt_name in opt_names:
    eff = results[opt_name]['test_acc'] / results[opt_name]['training_time']
    efficiencies[opt_name] = eff

sorted_eff = sorted(efficiencies.items(), key=lambda x: x[1], reverse=True)
for i, (opt, eff) in enumerate(sorted_eff):
    print(f"   {i+1}. {opt}: {eff:.6f} (acc/seg)")

print("\n3. OBSERVACIONES CLAVE:")
print(f"   • Mejor accuracy: {best_optimizer} ({float(best_acc):.4f})")
print(f"   • Más eficiente: {sorted_eff[0][0]}")
print(f"   • Convergencia más rápida: {min(opt_names, key=lambda x: results[x]['epochs_trained'])}")

print("4. RECOMENDACIONES POR USO:")
print("   ✅ Para máxima precisión: Adam/AdamW")
print("   ⏱️  Para entrenamiento rápido: RMSprop/Nadam")
print("   🔄 Para uso general: Adam (mejor balance)")
print("   📈 Para problemas sparse: Adagrad")

print("✅ PRÁCTICA 1 COMPLETADA Y OPTIMIZADA")

## Exportar Mejores Modelos

In [ ]:
# Guardar el mejor modelo para uso en prácticas posteriores
best_model = results[best_optimizer]['model']

# Evaluar modelo final
print(f"🎯 Modelo {best_optimizer} seleccionado como mejor")
print(f"   Test Accuracy: {results[best_optimizer]['test_acc']:.4f}")
print(f"   Tiempo entrenamiento: {results[best_optimizer]['training_time']:.2f}s")

# Resumen técnico
print("\n🔧 RESUMEN TÉCNICO:")
print(f"   - Modelo: CNN con {best_model.count_params():,} parámetros")
print(f"   - Arquitectura: Conv2D + BatchNorm + Dropout")
print(f"   - Regularización: Dropout (0.3, 0.5) + Early Stopping")
print(f"   - Data Augmentation: Configurado pero no usado en este experimento")
print(f"   - Optimizador ganador: {best_optimizer}")

print("✅ Modelo listo para integración en prácticas posteriores")